# Treino no dataset Cats & Dogs com e sem aumento dados e prétreino.

Neste Notobook iremos mostrar como usar o aumento dados para criar um classificador de imagens melhor.

Também mostraremos como usar redes pré-treinadas no Imagenet para se obter um melhor desempenho na tarefa final.

## Importação das bibliotecas

In [ ]:
!pip install neptune-client==0.9.1
!pip install pytorch_lightning

     |████████████████████████████████| 215kB 18.4MB/s 
     |████████████████████████████████| 829kB 53.3MB/s 
     |████████████████████████████████| 71kB 12.5MB/s 
     |████████████████████████████████| 163kB 57.3MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
     |████████████████████████████████| 133kB 60.9MB/s 
     |████████████████████████████████| 71kB 12.9MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.9.1-py2.py3-none-any.whl size=371716 sha256=76b62dc0066ead68c2c64b05a8373295379fffd7030098c9d4283aa2faac90ea
  Stored in directory: /root/.cache/pip/wheels/6e/d4/60/21005f18273566da0463587bed8c69cce7f300a404edf26df8
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=5827ee671cd1b99401929657103dbee5e8b4584f2f8d1339bcd20c9e46113a9a
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-cp37-none

In [ ]:
from collections import OrderedDict
import numpy as np
import neptune.new as neptune

import torch
from torch.utils.data import DataLoader
from torch import nn

import torchvision
from torchvision import transforms

import pytorch_lightning
from pytorch_lightning import LightningModule, Trainer

pytorch_lightning.utilities.seed.seed_everything(seed=123)

Global seed set to 123


123

## Dataset e dataloader

### Carregamento, criação dataset e do dataloader

In [ ]:
import os
import zipfile

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip


local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


--2021-05-08 01:07:41--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.128, 172.217.15.80, 172.217.13.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   201MB/s    in 0.3s    

2021-05-08 01:07:42 (201 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



Aqui criamos um dataset sem aumento de dados

In [ ]:
size_final = (224, 224)

transform_train = transforms.Compose([torchvision.transforms.Resize(size=size_final),
                                      torchvision.transforms.ToTensor()])
transform_valid = transform_train

In [ ]:
rootdir = '/tmp/cats_and_dogs_filtered'
train_dataset = torchvision.datasets.ImageFolder(
    rootdir + '/train/', transform=transform_train)

val_dataset = torchvision.datasets.ImageFolder(
    rootdir + '/validation/', transform=transform_valid)

In [ ]:
batch_size = 50

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print('Número de minibatches de trenamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))

x_train, y_train = next(iter(train_dataloader))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))

Número de minibatches de trenamento: 40
Número de minibatches de validação: 20

Dimensões dos dados de um minibatch: torch.Size([50, 3, 224, 224])
Valores mínimo e máximo dos pixels:  tensor(0.) tensor(1.)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>


Criação do modelo Pytorch Lightning

In [ ]:
class MyLightningModule(LightningModule):
    def __init__(self, hparams, run):
        super().__init__()

        print(hparams)
        self.hparams = hparams
        run['hparams'] = hparams  # Log to Neptune.
        self.criterion = nn.CrossEntropyLoss(reduction='none')

        # Carregando um modelo baseado na MobileNet V3.
        self.model = torchvision.models.mobilenet_v3_small(pretrained=hparams['pretrained'])

        # Modificando a ultima camada de classificação para 2 classes.
        n_features_in = self.model.classifier[3].weight.shape[1]
        self.model.classifier[3] = nn.Linear(n_features_in, 2)

    def forward(self, x):
        logits = self.model(x)
        preds = logits.argmax(dim=1)
        return logits, preds

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch

        # predict da rede
        logits = self.model(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        loss = batch_losses.mean()
        run['train/batch_loss'].log(loss)

        # O PL sempre espera um retorno nomeado 'loss' da training_step.
        return {'loss': loss, 'batch_losses': batch_losses}

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()

        run['train/loss'].log(avg_loss)
        return
  
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        
        # predict da rede
        logits, preds = self.forward(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        # calcula a acurácia
        batch_accuracy = (preds == y)
        
        # Retornamos as losses do batch para podermos fazer a média no validation_epoch_end.
        return {'batch_losses': batch_losses, 'batch_accuracy': batch_accuracy}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()
        accuracy = torch.stack([output['batch_accuracy'] for output in outputs]).float().mean()

        run['valid/loss'].log(avg_loss)
        run['valid/acuracy'].log(accuracy)

        self.log('valid_loss', avg_loss.item(), prog_bar=True)
        self.log('valid_accuracy', accuracy.item(), prog_bar=True)
  
    def test_step(self, val_batch, batch_idx):
        # A implementação deste método é opcional no Pytorch Lightning.
        x, y = val_batch
        
        # predict da rede
        logits, preds = self.forward(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        # calcula a acurácia
        batch_accuracy = (preds == y)
        
        # Retornamos as losses do batch para podermos fazer a média no validation_epoch_end.
        return {'batch_losses': batch_losses, 'batch_accuracy': batch_accuracy}

    def test_epoch_end(self, outputs):
        # A implementação deste método é opcional no Pytorch Lightning.
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()
        accuracy = torch.stack([output['batch_accuracy'] for output in outputs]).float().mean()

        run['valid/loss'].log(avg_loss)
        run['valid/acuracy'].log(accuracy)

        self.log('test_loss', avg_loss.item())
        self.log('test_accuracy', accuracy.item())

    def configure_optimizers(self):
        # Gradiente descendente
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.hparams['learning_rate'])
        # Aqui usamos um scheduler dummy pois o pytorch lightning original requer um.
        scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 1.0)
        return [optimizer], [scheduler]  # Forma de retorno para associar um otimizador a um scheduler.

## Treinamento sem aumento de dados

In [ ]:
hparams = {
    'max_epochs': 100,
    'learning_rate': 3e-4,
    'pretrained': False,
    'augmentation': False
}

# Insira seu api_token para logar os resultados do treino na sua conta do Neptune.
# Como obter seu API token do Neptune:
# https://docs.neptune.ai/administration/security-and-privacy/how-to-find-and-set-neptune-api-token

run = neptune.init(project='rodrigonogueira/aula-7-aug-pretrain', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhYzhlOTIxYy1lMGNhLTRiY2QtYTdjYi1jNWMyN2YxNzVhMTQifQ==')
run["sys/tags"].add([f'pre:{hparams["pretrained"]}', f'aug:{hparams["augmentation"]}'])

pl_model = MyLightningModule(hparams=hparams, run=run)
trainer = Trainer(max_epochs=hparams['max_epochs'], gpus=1)
trainer.fit(pl_model, train_dataloader, val_dataloader)

https://app.neptune.ai/rodrigonogueira/aula-7-aug-pretrain/e/AUL3-19
{'max_epochs': 100, 'learning_rate': 0.0003, 'pretrained': False, 'augmentation': False}


AttributeError: ignored

## Treinamento com aumento de dados

Abaixo fazemos várias transformações na imagem, como mudança nas cores e recortes aleatórios.

In [ ]:
size_large = (256, 256)
size_final = (224, 224)

transform_train = transforms.Compose([
    transforms.Resize(size=size_large),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomAffine(3, translate=None, scale=(0.95, 1.05), shear=None, resample=False, fillcolor=2),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=size_final),
    transforms.ToTensor()])

# Não funciona se fizermos train_dataloader.dataset.transform, portanto temos que recriar o Dataset e Dataloader.
train_dataset = torchvision.datasets.ImageFolder(
    rootdir + '/train/', transform=transform_train)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1315: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1329: UserWarning: Argument fillcolor is deprecated and will be removed since v0.10.0. Please, use fill instead
  "Argument fillcolor is deprecated and will be removed since v0.10.0. Please, use fill instead"


In [ ]:
hparams = {
    'max_epochs': 100,
    'learning_rate': 3e-4,
    'pretrained': False,
    'augmentation': True
}
run = neptune.init(project='rodrigonogueira/aula-7-aug-pretrain', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhYzhlOTIxYy1lMGNhLTRiY2QtYTdjYi1jNWMyN2YxNzVhMTQifQ==')
run["sys/tags"].add([f'pre:{hparams["pretrained"]}', f'aug:{hparams["augmentation"]}'])

pl_model = MyLightningModule(hparams=hparams, run=run)
trainer = Trainer(max_epochs=hparams['max_epochs'], gpus=1)
trainer.fit(pl_model, train_dataloader, val_dataloader)

https://app.neptune.ai/rodrigonogueira/aula-7-aug-pretrain/e/AUL3-13


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | model     | MobileNetV3      | 1.5 M 
-----------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.080     Total estimated model params size (MB)


1

## Com modelo pré-treinado no ImageNet

In [ ]:
hparams = {
    'max_epochs': 20,
    'learning_rate': 3e-4,
    'pretrained': True,
    'augmentation': True
}
run = neptune.init(project='rodrigonogueira/aula-7-aug-pretrain', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhYzhlOTIxYy1lMGNhLTRiY2QtYTdjYi1jNWMyN2YxNzVhMTQifQ==')
run["sys/tags"].add([f'pre:{hparams["pretrained"]}', f'aug:{hparams["augmentation"]}'])

pl_model = MyLightningModule(hparams=hparams, run=run)
trainer = Trainer(max_epochs=hparams['max_epochs'], gpus=1)
trainer.fit(pl_model, train_dataloader, val_dataloader)

https://app.neptune.ai/rodrigonogueira/aula-7-aug-pretrain/e/AUL3-14


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | model     | MobileNetV3      | 1.5 M 
-----------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.080     Total estimated model params size (MB)


1

Em suma, as acurácias no dataset de validação foram:
- sem aumento de dados: ~60%
- com aumento de dados: ~80%
- com aumento e prétreino no ImageNet: ~96%